In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
version_dir = "version/" + timestamp 

os.makedirs(version_dir)

In [13]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["TRAIN_SPLIT"] = 100000
param_list["BATCH_SIZE"] = 256
param_list["EPOCHS"] = 100
param_list["BUFFER_SIZE"] = 200000
param_list["EVALUATION_INTERVAL"] = 300
param_list["VAL_STEPS"] = 50

with open("version/{0}/param_list.json".format(timestamp), 'w') as j:
    json.dump(param_list, j, indent=4)

In [5]:
pattern_length = 4

dataset = pd.read_csv("data/SEG_train_dataset_{}.csv".format(pattern_length))
dataset.head()

,scaled,delta,t,t+1
0,0.398202,-1587653544,95444983808,93857330264
1,0.895775,6172925864,87674650304,93847576168
2,0.103254,-6187909720,93857313352,87669403632
3,0.103253,-6187918552,93857325056,87669406504
4,0.102577,-6198466584,93857330264,87658863680


In [6]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [8]:
x_train, y_train = generate_timeseries(dataset["scaled"].values, 0, param_list["TRAIN_SPLIT"], param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])
x_val, y_val = generate_timeseries(dataset["scaled"].values, param_list["TRAIN_SPLIT"], None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat()
train_data = train_data.cache().shuffle(param_list["BUFFER_SIZE"]).batch(param_list["BATCH_SIZE"])
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).repeat()
val_data = val_data.cache().batch(param_list["BATCH_SIZE"])

In [9]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, return_sequences=True, input_shape=x_train.shape[-2:]))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Activation("relu"))

model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

In [10]:
model_history = model.fit(train_data, epochs=param_list["EPOCHS"], steps_per_epoch=param_list["EVALUATION_INTERVAL"], validation_data=val_data, validation_steps=param_list["VAL_STEPS"], callbacks=[tensorboard_callback])
model.save("version/{0}/model.h5".format(timestamp))

Epoch 1/100
300/300 [==============================] - 3s 9ms/step - loss: 0.0169 - mse: 0.0169 - mae: 0.0752 - val_loss: 0.0094 - val_mse: 0.0094 - val_mae: 0.0337
Epoch 2/100
300/300 [==============================] - 2s 7ms/step - loss: 0.0103 - mse: 0.0103 - mae: 0.0530 - val_loss: 0.0092 - val_mse: 0.0092 - val_mae: 0.0352
Epoch 3/100
300/300 [==============================] - 2s 7ms/step - loss: 0.0094 - mse: 0.0094 - mae: 0.0481 - val_loss: 0.0089 - val_mse: 0.0089 - val_mae: 0.0334
Epoch 4/100
300/300 [==============================] - 2s 7ms/step - loss: 0.0090 - mse: 0.0090 - mae: 0.0460 - val_loss: 0.0088 - val_mse: 0.0088 - val_mae: 0.0334
Epoch 5/100
300/300 [==============================] - 2s 7ms/step - loss: 0.0090 - mse: 0.0090 - mae: 0.0450 - val_loss: 0.0088 - val_mse: 0.0088 - val_mae: 0.0350
Epoch 6/100
300/300 [==============================] - 2s 7ms/step - loss: 0.0089 - mse: 0.0089 - mae: 0.0435 - val_loss: 0.0087 - val_mse: 0.0087 - val_mae: 0.0346
Epoch 7/10